# core

> This is where users start to interact with the underlying LLM and agents.

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
# install dependencies
# %pip install replicate
# %pip install gradio
# %pip install langchain

In [ ]:
#| export
# import langchain

import os
import llm_QA


In [ ]:
#| export

model = llm_QA.LLM("meta-llama", "TogetherAI")

In [ ]:
#| hide
# interact with the model directly to test
messages = [
  {
    "role": "user",
    "content": "what are large language models?"
  }
]

response = model.get_llm_response(messages)
print(response)

Error: can only concatenate str (not "NoneType") to str
None


In [ ]:
#| hide
# interact with the model directly to test multimodal inputs
from PIL import Image
import matplotlib.pyplot as plt

def display_local_image(image_path):
    img = Image.open(image_path)
    plt.imshow(img)
    plt.axis('off')
    plt.show()

display_local_image("images/a_colorful_llama_doing_ai_programming.jpeg")

import base64

def encode_image(image_path):
  with open(image_path, "rb") as img:
    return base64.b64encode(img.read()).decode('utf-8')

base64_image = encode_image("images/a_colorful_llama_doing_ai_programming.jpeg")

messages = [
  {
    "role": "user",
    "content": [
      {
        "type": "text",
        "text": "describe the image!"
      },
      {
        "type": "image_url",
        "image_url": {
          "url": f"data:image/jpeg;base64,{base64_image}"
        }
      }
    ]
  },
]

response = model.get_llm_response(messages)
print(response)

In [ ]:
#| export
# A chatbot that allows users to interact with an LLM
import gradio as gr

def taskRouter(input, history):
    prompt = input["text"]
    files = input["files"]
    messages = [history,
        {
            "role": "user",
            "content": prompt
        }
    ]

    response = model.get_llm_response(messages)
    #history = [messages, 
    #    {
    #        "role": "assistant",
    #        "content": response
    #    }
    #]
    return response

def vote(data: gr.LikeData):
    if data.liked:
        print("You upvoted this response: " + data.value["value"])
    else:
        print("You downvoted this response: " + data.value["value"])

with gr.Blocks() as sciChat:
    chatbot = gr.Chatbot(
        height=300,
        type="messages",
        placeholder="<strong>Your Personal Science AI Assistant</strong><br>Ask Me Anything")
    chatbot.like(vote, None, None)
    gr.ChatInterface(
        fn=taskRouter,
        type="messages",
        chatbot=chatbot,
        textbox=gr.MultimodalTextbox(placeholder="How can I help?", container=False, scale=7),
        title="SciAI Assistant",
        description="Ask me any question or carry out a task",
        theme="soft",
        examples=[{"text": "Hello", "files": []},{"text": "What do you see in this image?", "files": []}],
        cache_examples=True,
        multimodal=True
    )
    
sciChat.launch(share=True)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()